# TPCAV detailed usage

This tutorial goes through the detailed steps of TPCAV

In [1]:
# data download
!mkdir data/
!wget https://hgdownload.gi.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz -P data/
!gunzip data/hg38.fa.gz
!wget https://raw.githubusercontent.com/seqcode/TPCAV/main/data/motif-clustering-v2.1beta_consensus_pwms.test.meme -P data/

import pyfaidx
pyfaidx.Fasta('data/hg38.fa')

--2026-01-20 16:34:56--  https://hgdownload.gi.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.gi.ucsc.edu (hgdownload.gi.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.gi.ucsc.edu (hgdownload.gi.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘data/hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  38.6MB/s    in 26s     

2026-01-20 16:35:23 (36.3 MB/s) - ‘data/hg38.fa.gz’ saved [983659424/983659424]

--2026-01-20 16:35:50--  https://raw.githubusercontent.com/seqcode/TPCAV/main/data/motif-clustering-v2.1beta_consensus_pwms.test.meme
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3504 (3.4K) [text/plain]
Saving to: 

Fasta("data/hg38.fa")

Since TPCAV is a concept based attribution method, the first step is to construct candidate concepts for testing, each concept is a set of input examples that share a similar pattern (e.g. motif).

Assume we start with a dummy model that has two linear layers taking 1024bp long one-hot coded DNA input for predicting a scalar value.

In [2]:
import torch

class DummyModelSeq(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(1024, 1)
        self.layer2 = torch.nn.Linear(4, 1)

    def forward(self, seq):
        y_hat = self.layer1(seq)
        y_hat = y_hat.squeeze(-1)
        y_hat = self.layer2(y_hat)
        return y_hat

Now we start constructing concepts, a concept is basically an iterator of the inputs that represent a prominent pattern. TPCAV provides a class `ConceptBuilder` to construct common concepts in genomics field, here we use it to construct motif concepts, defined as sets of random genomic sequences inserted by motif instances.

There are 5 motif concepts built in this case given the test meme file

In [3]:
from pathlib import Path
from tpcav.concepts import ConceptBuilder

motif_path = Path("data") / "motif-clustering-v2.1beta_consensus_pwms.test.meme"

# create concept builder to generate concepts
builder = ConceptBuilder(
    genome_fasta="data/hg38.fa",
    input_window_length=1024,
    bws=None,
    num_motifs=12,
    include_reverse_complement=True,
    min_samples=1000,
    batch_size=8,
)
# use random regions as control  
builder.build_control()
# use meme motif PWMs to build motif concepts, one concept per motif
builder.add_meme_motif_concepts(str(motif_path))

/home/jmy5455/miniforge3/envs/test/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Concept(1, 'AC0001:GATA-PROP:GATA'),
 Concept(2, 'AC0002:PROP-ALX:Homeodomain'),
 Concept(3, 'AC0003:HNF1A-HNF1B:Homeodomain'),
 Concept(4, 'AC0004:ZSCAN:C2H2_ZF'),
 Concept(5, 'AC0005:POU3F-POU1F:Homeodomain,POU')]

Concept constructed by `ConceptBuilder` contains an iterator of two things: fasta sequences strings and array of bigwig signal tracks. If your model takes different formats of inputs, you can provide a transformation function to fit your model. Below is the example of obtaining one hot coded DNA sequence inputs.

In [4]:
# each batch is a tuple of a fasta sequence list and an array of bigwig signals
next(iter(builder.concepts[0].data_iter))

(['ATTTTACTAACTGCCACACTGCATAGCTTTTATAATTAACTTATTAGATTTACGGGAGAGAGGCACCAGAGTGGATTACAACACAGGACTGACCTTTTACCACCCGGCTGAAGCTCGAAAAACAAAACCAAATCCCATCTTACTAACCATACCCTAACTCCCTTAAGGAAATACCCCAGAATACAAGTTTGTTTGTTTTTTAAGATGGTTTTCCTTCCCTGAGAGAACAATCCTGAAACTTAAGTCAATAAAGATAAGGTCATTATTAAGCATTTTGCCTCCTCGGGTCAGACTTGAGGCCTTTTCCCCCCTTTCTTCTCGCTAAAGGCATTATTGCCTTAGATCACGGTGCATCCTAGGCAGGTTATGCCATTTTAATCTCATAAATATGAAAATAAAAGACATAATCAGAATAATGTAGTTACTTTTTATTTTTAAACCAAAGGAAGCCAAAATAGCAACAGAGAAGAAAGGAAATGTAAGGCACTTAACTCTGGGGCCAAGGTGCATTTGCCCTAGTTCTTTTTTTACTAATCTACTTACTTTTCTTCCTCCCCATTCCACAGCACTCAGCATCTTCTATCTTGTCTCCCTTGGTGTAGCTCTAGTGTTCTAATTCTATCCCATCTTCTTCTACTGAAATAATGAACTTTTTAAAGAGAAAATTAAGTGAATGTCTTTGTAGCAATGAACCTCCCCCTCTCGAGAATAAAAGGGAGCTTTGGAGATAAGGCAATGCTTAACCCAGTGAACGAATTGCTTACTGATGGGGTCAGCTAACCACATTATAGGGTGGCAGAGCTGACTCTGCCTACTGATATCGGCCATGTTAATTACCGTAATTATTATATACTCAACATCTATATAAATGGATTATTGTTAGCTTGTTTCTATTTTATAACAGCCCTGAAATGTAAAATGGTTTTATTTTATTTATTTATTCTAACATTGTTATTTTTTTGAGATACCAAGGTTATCTGTCGCCCAGGCTGGAGTG

In [5]:
from tpcav import helper

# apply transformation function to obtain one-hot encoded sequences
def transform_fasta_to_one_hot_seq(seq, chrom):
    # `seq` is a list of fasta sequences
    # `chrom` is a numpy array of bigwig signals of shape [-1, # bigwigs, len]
    return (helper.fasta_to_one_hot_sequences(seq),) # it has to return a tuple of inputs, even if there is only one input
builder.apply_transform(transform_fasta_to_one_hot_seq)

In [6]:
# after transformation each batch is one-hot coded DNA arrays wrapped in tuple
next(iter(builder.concepts[0].data_iter))

(tensor([[[1., 0., 0.,  ..., 0., 1., 1.],
          [0., 0., 0.,  ..., 1., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 1.,  ..., 0., 0., 0.]],
 
         [[1., 0., 0.,  ..., 0., 1., 1.],
          [0., 0., 0.,  ..., 1., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 1.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 1., 0., 1.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 1., 0., 1.]],
 
         [[0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 1., 0., 1.]],
 
         [[0., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 1., 1., 0.],
   

After concept construction, now it's time to train the linear classifier for every concept in your model. You need to wrap your model instance by `TPCAV` class with the name of the layer for interpretation provided, then we apply PCA transformation given the concepts we just built to decorrelate the embedding space to solve the correlated redundant feature issue.

In [7]:
from tpcav.tpcav_model import TPCAV
# create TPCAV model on top of your model
tpcav_model = TPCAV(DummyModelSeq(), layer_name="layer1")
# fit PCA on sampled all concept activations
tpcav_model.fit_pca(
    concepts=builder.all_concepts(),
    num_samples_per_concept=10,
    num_pc="full",
)
# you can save the tpcav model in case of future use
torch.save(tpcav_model, "data/tmp_tpcav_model.pt")

INFO:tpcav.tpcav_model:Sampled 10 activations from concept AC0001:GATA-PROP:GATA
INFO:tpcav.tpcav_model:Sampled 10 activations from concept AC0002:PROP-ALX:Homeodomain
INFO:tpcav.tpcav_model:Sampled 10 activations from concept AC0003:HNF1A-HNF1B:Homeodomain
INFO:tpcav.tpcav_model:Sampled 10 activations from concept AC0004:ZSCAN:C2H2_ZF
INFO:tpcav.tpcav_model:Sampled 10 activations from concept AC0005:POU3F-POU1F:Homeodomain,POU
INFO:tpcav.tpcav_model:Sampled 10 activations from concept random_regions


In [8]:
# now tpcav_model contains the necessary parameters for PCA transformation
tpcav_model.pca_inv

tensor([[-0.7210,  0.1799,  0.6242,  0.2412],
        [ 0.0054, -0.7056, -0.0630,  0.7057],
        [-0.1245,  0.6124, -0.5386,  0.5651],
        [-0.6816, -0.3077, -0.5624, -0.3527]], device='cuda:0')

Using the fitted TPCAV model, we can train the concept activation vectors (CAVs) on the concepts we built

In [9]:
from tpcav.cavs import CavTrainer

# create trainer for computing CAVs
cav_trainer = CavTrainer(tpcav_model, penalty="l2")
# set control concept for CAV training
cav_trainer.set_control(builder.control_concepts[0], num_samples=100)
# train CAVs for all concepts
cav_trainer.train_concepts(
    builder.concepts, 100, output_dir="data/cavs/", num_processes=2
)

INFO:tpcav.cavs:Submitted CAV training for concept AC0001:GATA-PROP:GATA
INFO:tpcav.cavs:Submitted CAV training for concept AC0002:PROP-ALX:Homeodomain
INFO:tpcav.cavs:Submitted CAV training for concept AC0003:HNF1A-HNF1B:Homeodomain
INFO:tpcav.cavs:Submitted CAV training for concept AC0004:ZSCAN:C2H2_ZF
INFO:tpcav.cavs:Submitted CAV training for concept AC0005:POU3F-POU1F:Homeodomain,POU
INFO:tpcav.cavs:Best Params: {'alpha': 0.0001} | Iterations: 15
INFO:tpcav.cavs:Best Params: {'alpha': 1e-06} | Iterations: 14
INFO:tpcav.cavs:[train] Accuracy: 0.5312
INFO:tpcav.cavs:[train] Accuracy: 0.5312
INFO:tpcav.cavs:[val] Accuracy: 0.5000
INFO:tpcav.cavs:[val] Accuracy: 0.4000
INFO:tpcav.cavs:[test] Accuracy: 0.4500
INFO:tpcav.cavs:[test] Accuracy: 0.3500
INFO:tpcav.cavs:Best Params: {'alpha': 0.0001} | Iterations: 13
INFO:tpcav.cavs:Best Params: {'alpha': 0.0001} | Iterations: 11
INFO:tpcav.cavs:[train] Accuracy: 0.4875
INFO:tpcav.cavs:[train] Accuracy: 0.5062
INFO:tpcav.cavs:[val] Accuracy:

In [10]:
# after training, the cavs are stored
cav_trainer.cav_weights

{'AC0001:GATA-PROP:GATA': tensor([-7.3748e+00, -7.7315e+00, -1.4255e+00, -9.0382e+00,  6.0171e-05,
          1.7808e-05, -9.3377e-06, -2.0687e-05]),
 'AC0002:PROP-ALX:Homeodomain': tensor([ 2.2075e+01,  1.6838e+02,  8.1769e+01, -8.9027e+01,  1.4293e-04,
          1.2089e-04, -1.9308e-04, -1.2634e-04]),
 'AC0003:HNF1A-HNF1B:Homeodomain': tensor([ 2.0150e+01,  1.4207e+01,  9.9167e+00, -1.0596e+01,  4.3837e-06,
          1.0020e-05, -3.1145e-05,  3.5364e-06]),
 'AC0004:ZSCAN:C2H2_ZF': tensor([ 1.1846e+01, -1.0493e+01,  2.5760e+00, -9.7888e+00,  2.4079e-05,
          1.5101e-05, -2.6995e-05,  6.7725e-05]),
 'AC0005:POU3F-POU1F:Homeodomain,POU': tensor([ 7.7956e+01, -9.6370e+01, -3.6764e+01, -9.0410e+01,  3.4096e-04,
         -9.1233e-05, -8.2176e-05, -5.8771e-05])}

Finally, we can compute the layer attribution scores on the test regions against control regions, and obtain the final TPCAV score for each concept

In [11]:
# create input regions and baseline regions for attribution
random_regions_1 = helper.random_regions_dataframe(
    "data/hg38.fa.fai", 1024, 100, seed=1
)
random_regions_2 = helper.random_regions_dataframe(
    "data/hg38.fa.fai", 1024, 100, seed=2
)
# create iterators to yield one-hot encoded sequences from the region dataframes
def pack_data_iters(df):
    seq_fasta_iter = helper.dataframe_to_fasta_iter(
        df, "data/hg38.fa", batch_size=8
    )
    seq_one_hot_iter = (
        helper.fasta_to_one_hot_sequences(seq_fasta)
        for seq_fasta in seq_fasta_iter
    )
    return zip(seq_one_hot_iter, )
# compute layer attributions given the iterators of testing regions and control regions
attributions = tpcav_model.layer_attributions(
    pack_data_iters(random_regions_1), pack_data_iters(random_regions_2)
)["attributions"]
# compute TPCAV scores for the concept
cav_trainer.tpcav_score_all_concepts_log_ratio(attributions)

/home/jmy5455/miniforge3/envs/test/lib/python3.14/site-packages/captum/attr/_core/deep_lift.py:294: UserWarning: Input Tensor 1 did not already require gradients, required_grads has been set automatically.
  gradient_mask = apply_gradient_requirements(inputs_tuple)
/home/jmy5455/miniforge3/envs/test/lib/python3.14/site-packages/captum/log/dummy_log.py:39: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  return func(*args, **kwargs)


{'AC0001:GATA-PROP:GATA': -3.1986731175506815,
 'AC0002:PROP-ALX:Homeodomain': 1.4744016286301573,
 'AC0003:HNF1A-HNF1B:Homeodomain': 3.198673117550681,
 'AC0004:ZSCAN:C2H2_ZF': -0.03922071315328127,
 'AC0005:POU3F-POU1F:Homeodomain,POU': 0.4795730802618863}